In [ ]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
commentaire_negatif_att_set_hubiscus = dataiku.Dataset("commentaire_negatif_att_set_hubiscus")
commentaire_negatif_att_set_hubiscus_df = commentaire_negatif_att_set_hubiscus.get_dataframe()

liste_commentaire = list(commentaire_negatif_att_set_hubiscus_df['commentaire'])
liste_nom = list(commentaire_negatif_att_set_hubiscus_df['nom'])

# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

#commentaire_negatif_tf_df = commentaire_negatif_att_set_hubiscus_df # For this sample code, simply copy input to output


# Write recipe outputs
#commentaire_negatif_tf = dataiku.Dataset("commentaire_negatif_tf")
#commentaire_negatif_tf.write_with_schema(commentaire_negatif_tf_df)

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: MARKDOWN
# *wordcloud*

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# pour wordcloud
final_stopwords_list = list(fr_stop)
vectorizer = TfidfVectorizer(stop_words=final_stopwords_list) #stop_words=final_stopwords_list
transformer=TfidfTransformer()
tfidf=transformer.fit_transform(vectorizer.fit_transform(liste_commentaire))
word=vectorizer.get_feature_names()
weight=tfidf.toarray()

liste_dic_all = []
for i in range(len(weight)):#打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for便利某一类文本下的词语权重  
        #print (u"-------这里输出第",i,u"类文本的词语tf-idf权重------" ) 
        liste_dic = []
        dic = {}
        for j in range(len(word)):  
           
            #dic.update(word[j]:weight[i][j])
            dic[word[j]] = weight[i][j]
        liste_dic.append(dic)
        liste_dic_all.append(liste_dic)

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
l =[] #tous les mots
for i in range(len(liste_dic_all)):
#liste_1 = liste_dic_all[5]
    for i in liste_dic_all[i]:
        for key,value in i.items():
            if(value > 0):
           # if 'attend' in key:
                #print (key,value)
                l.append(key)
print(len(l))

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: MARKDOWN
# *mots 4 gram*

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# pour des mots 4 gram
final_stopwords_list = list(fr_stop)
vectorizer = TfidfVectorizer(ngram_range=(4,4)) #stop_words=final_stopwords_list
transformer=TfidfTransformer()
tfidf=transformer.fit_transform(vectorizer.fit_transform(liste_commentaire))
word=vectorizer.get_feature_names()
weight=tfidf.toarray()

liste_dic_all = []
for i in range(len(weight)):  
       
        liste_dic = []
        dic = {}
        for j in range(len(word)):  
           
            #dic.update(word[j]:weight[i][j])
            dic[word[j]] = weight[i][j]
        liste_dic.append(dic)
        liste_dic_all.append(liste_dic)
            #print(dic)
            #print(word[j],weight[i][j])  
#print(liste_dic_all[0])

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
l_word =[] 
for f in range(len(liste_nom)):
    for i in liste_dic_all[f]:
        dic = {}
        dic['nom'] = str(liste_nom[f])
        l=[]
        for key,value in i.items():
            
            if(value == max(i.values())):
                l.append(key)
        dic['tfidf_mot'] = l
        l_word.append(dic)
print(l_word[0])

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
new_df = pd.DataFrame(l_word)
commentaire_negatif_tf = dataiku.Dataset("commentaire_negatif_tf")
commentaire_negatif_tf.write_with_schema(new_df)